In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Input, Bidirectional, Conv1D, GRU, LSTM,TimeDistributed, Flatten
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.utils import plot_model
from tqdm.auto import tqdm

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')

# Testing data
test_labels = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv')
test = test.merge(test_labels, on="id")

# Label column names
labels = list(train.columns[2:])

# GloVe embedding
glove_file = '../input/glove840b300dtxt/glove.840B.300d.txt'

# Make output directory
out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
# Drop all rows with -1 in label values
drop_idxs = test[
    (test.toxic == -1) | (test.severe_toxic == -1) | (test.obscene == -1) | 
    (test.threat == -1) | (test.insult == -1) | (test.identity_hate == -1)
].index
test = test.drop(drop_idxs, axis="rows")

In [ ]:
# Features
X_train = train["comment_text"].str.lower()
X_test = test["comment_text"].str.lower()

# Labels
y_train = train[labels].values
y_test = test[labels].values

# Training data
print(f"X_train.shape = {X_train.shape}")
print(f"y_train.shape = {y_train.shape}")
# Testing data
print(f"X_test.shape = {X_test.shape}")
print(f"y_test.shape = {y_test.shape}")

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(16, 6))
for col, ax in zip(labels, axes.flatten()):
    ax.hist(train[col], label="train")
    ax.hist(test[col], label="test")
    ax.set_title(f"{col}")
    ax.set_yscale("log")
    ax.legend()
plt.tight_layout()
plt.savefig(f"{out_dir}/class-label-distribution.jpg", dpi=300)
plt.show()

In [ ]:
max_features = 150000
maxlen = 200
embed_size = 300

tok = text.Tokenizer(num_words=max_features, lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train = tok.texts_to_sequences(X_train)
X_test = tok.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
embeddings_index = {}
with open(glove_file, encoding='utf8') as f:
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# nlp = spacy.load('en'); # Depricated
import en_core_web_sm
nlp = en_core_web_sm.load()

word_index = tok.word_index

num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.random.randn(num_words, embed_size)/4
kk = 0
moo = 0
for word, i in tqdm(word_index.items()): 
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        kk += 1
    else: 
        for x in nlp(word,disable=['parser', 'ner']):
            embedding_vector = embeddings_index.get(x.lemma_)
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector 
                kk += 1
                break

In [ ]:
# main model
model_input = Input(shape=(maxlen,))
model = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(model_input)
model =  Bidirectional (LSTM (256,return_sequences=True,dropout=0.25),merge_mode='concat')(model)
model =  Bidirectional (LSTM (128,return_sequences=True,dropout=0.25),merge_mode='concat')(model)
avg_pool = GlobalAveragePooling1D()(model)
max_pool = GlobalMaxPooling1D()(model)
model = concatenate([avg_pool, max_pool]) 
model = Dense(100,activation='relu')(model)
preds = Dense(6,activation='sigmoid')(model)
model = Model(model_input,preds,name="project_model")


In [ ]:
es = EarlyStopping(monitor='accuracy', patience = 2, restore_best_weights=True)

In [ ]:
lr_anneal = ReduceLROnPlateau(monitor = 'accuracy', patience=2, factor=0.2, min_lr=1e-7,restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
plot_model(model)


In [ ]:
batch_size = 128
epochs = 20
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=2e-4),metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,callbacks = [lr_anneal,es])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(12, 6))

# summarize history for accuracy
ax[0].plot(history.history['accuracy'])
ax[0].set_title('project model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].set_xticks(np.arange(0, epochs))
# summarize history for loss
ax[1].plot(history.history['loss'])
ax[1].set_title('project model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].set_xticks(np.arange(0, epochs))
plt.savefig(f"{out_dir}/project-model-training.jpg", dpi=300)
plt.show()

In [ ]:
y_pred = model.predict(x_test, batch_size=1024, verbose=1)

In [ ]:
tests = pd.DataFrame(y_test, columns=labels)
preds = pd.DataFrame(y_pred, columns=labels).round() # We have to round the probabilities (0-1 inclusive) to binary (0 or 1)

acc_dict_proj = {}
f1s_dict_proj = {}
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(12, 6))
for col, ax in zip(labels, axes.flatten()):
    true_label = tests[col]
    prediction = preds[col]
    # Record accuracy and F1 score
    acc = acc_dict_proj[col] = accuracy_score(true_label, prediction)
    f1 = f1s_dict_proj[col] = f1_score(true_label, prediction)
    # Confusion matrix
    (tn, fp), (fn, tp) = conf_matt = confusion_matrix(true_label, prediction)
    sns.heatmap(conf_matt, annot=True, cbar=False, fmt=".2f", ax=ax)
    ax.set_title(f"{col}\naccuracy: {acc:.3f}\nF1-score: {f1:.3f}")
    ax.set_xlabel("Prediction")
    ax.set_ylabel("True")
plt.tight_layout()
plt.savefig(f"{out_dir}/project-model-confusion-matrix.jpg", dpi=300)
plt.show()

In [ ]:
# initialize evaluation metric comparison dataframe
eval_df = pd.DataFrame(
    index=["paper_model","project_model"],
    columns=["acc_mean", "acc_sd", "f1_mean", "f1_sd"]
)

In [ ]:
accuracies_proj = list(acc_dict_proj.values())
mu = eval_df["acc_mean"]["project_model"] = np.mean(accuracies_proj)
sd = eval_df["acc_sd"]["project_model"] = np.std(accuracies_proj)
print(f"Avg accuracy: {mu:.3f} ± {sd:.3f}")

# f1-score
f1_scores_proj = list(f1s_dict_proj.values())
mu = eval_df["f1_mean"]["project_model"] = np.mean(f1_scores_proj)
sd = eval_df["f1_sd"]["project_model"] = np.std(f1_scores_proj)
print(f"Avg F1-score: {mu:.3f} ± {sd:.3f}")